In [ ]:
## obs: sera importante informar o caminho das cidade / loja para configurar os webdriver.

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time


options = Options()
options.add_argument("--incognito")
options.add_argument("--window-size=2560,1440")
options.add_argument("--start-maximized")
#options.add_argument('--headless=new') ## modo oculto.
driver = webdriver.Chrome(options=options)

# Wait for initialize, in seconds
wait = WebDriverWait(driver, 10)

driver.get("https://mercado.carrefour.com.br/#crfimt=home|shop|seletor|mercado_4p_161020|2")

##driver.quit()

In [27]:
button_insira_cep = driver.find_elements(By.XPATH,"//button[ text()='Insira seu CEP']")
button_insira_cep[1].click()

button_retire_na_loja = driver.find_elements(By.XPATH,"//span[@class='w-[7rem] mx-[.75rem]' ]")
button_retire_na_loja[1].click()

button_selecione_cidade = driver.find_elements(By.XPATH,"//select[@id='selectCity' ]//option")[2:]
contador = 0
lista_cidades ={}
for i in button_selecione_cidade:
    lista_cidades[i.text] = contador
    contador += 1

In [28]:
button_selecione_cidade[ lista_cidades['SÃO PAULO - SP'] ].click()

#aguarda ate o botao loja estar visivel:
wait.until(EC.visibility_of_element_located((By.XPATH,"//div[@class='grid grid-flow-col']//h3")))

button_loja = driver.find_elements(By.XPATH,"//div[@class='grid grid-flow-col']//h3")

lista_de_lojas = {}
contador = 0
for i in button_loja:
    lista_de_lojas[i.text] = contador
    contador += 1

button_loja[ lista_de_lojas['Hipermercado Pamplona'] ].click()


In [29]:
## parte em que os produtos são pesquisados no site :

caixa_pesquisa = driver.find_elements(By.XPATH,'//input')[0]
acoes = ActionChains(driver)
acoes.move_to_element(caixa_pesquisa).click().send_keys("cafe").send_keys(Keys.ENTER).perform()


In [ ]:
## é necessario ajustar a pagina apenas uma vez para mostrar 60 itens por pagina:

button_quantidade_produtos_pagina = driver.find_element(By.XPATH,'//button[@class="flex justify-between items-center color-[#333333] p-[7px]"]')
button_exibe_produtos = driver.find_element(By.XPATH,'//button[text()="60"]')
acoes.move_to_element(button_quantidade_produtos_pagina).click().perform()
acoes.move_to_element(button_exibe_produtos).click().perform()

In [ ]:
lista_produtos = driver.find_elements(By.XPATH,'//ul[@class="grid grid-cols-2 xl:grid-cols-5 md:grid-cols-4"]//li//article//section//h3//a')
dicionario_produtos = {}
for i in lista_produtos:
    dicionario_produtos[i.get_attribute('href')] = i.text  

id_produtos = []
for i in dicionario_produtos.keys():
   id_produtos.append( i.replace('/p','').split('-')[-1:] )


pd.DataFrame({'Id Produto':id_produtos,'Nome':dicionario_produtos.values(),'Link':dicionario_produtos.keys()})

In [56]:
driver.quit()

In [ ]:
## codigo que puxa toda a relacao de Estado/Loja disponiveis no site do carrefour mercado:

base = []
for cidade in list(lista_cidades.keys()):
    button_muda_loja = driver.find_element(By.XPATH,'//span[@class="flex hover:underline hover:underline-offset-2 font-bold col-span-2"]')
    button_muda_loja.click()
    time.sleep(2)
    button_selecione_cidade_2 = driver.find_elements(By.XPATH,"//select[@id='selectCity' ]//option")[2:]
    button_selecione_cidade_2[ lista_cidades[ cidade ] ].click()
    time.sleep(2)
    button_loja = driver.find_elements(By.XPATH,"//div[@class='grid grid-flow-col']//h3")
    lista_de_lojas = {}
    contador = 0
    for i in button_loja:
        lista_de_lojas[i.text] = contador
        contador += 1
    button_loja[0].click()
    time.sleep(2)
    cidade_ = [cidade for i in range(len(button_loja))]
    data = pd.DataFrame({'cidade':cidade_,'Lojas':list(lista_de_lojas.keys())})
    base.append(data)
    print(f'cidade analisada: {cidade}')

## gera a base de dados:
base_de_lojas = pd.concat(base,ignore_index=True)
base_de_lojas.to_excel("base de lojas.xlsx")

In [15]:
driver.quit()

In [54]:
button_loja = driver.find_elements(By.XPATH,"//div[@class='grid grid-flow-col']//h3")

In [ ]:
lista_de_lojas = {}
contador = 0
for i in button_loja:
    lista_de_lojas[i.text] = contador
    contador += 1
lista_de_lojas



In [56]:
button_loja[ lista_de_lojas['Carrefour Bairro Morumbi'] ].click()